# Using the PlayerData API with Direct

This notebook demonstrates how to use the PlayerData API with Direct Queries to query session details and raw data.

## Prerequisites

Before using this notebook, you will need to:
- Create a virtual environment
- Install all dependencies from the `pyproject.toml` file

## Authentication

First, we need to authenticate with the PlayerData API. There are three authentication methods available:

1. **Client Credentials Flow** (shown in this example)
2. **Authorization Code Flow**
3. **Authorization Code Flow with PKCE**

To use a different authentication method, change the `authentication_type` parameter to `AuthenticationType.AUTHORIZATION_CODE_FLOW` or `AuthenticationType.AUTHORIZATION_CODE_FLOW_WITH_PKCE` and provide the required parameters.

In [ ]:
from playerdatapy.gqlauth import GraphqlAuth
from playerdatapy.gqlauth import AuthenticationType
from playerdatapy.gqlclient import Client
from playerdatapy.constants import API_BASE_URL

# Add your client id, client secret and one of your club ids
CLIENT_ID = "your_client_id"
CLIENT_SECRET = "your_client_secret"
CLUB_ID = "your_club_id"

# Example usage of the GraphqlClient class.
auth = GraphqlAuth(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    type=AuthenticationType.CLIENT_CREDENTIALS_FLOW,
)

# Create a Client instance.
client = Client(
    url=f"{API_BASE_URL}/api/graphql",
    headers={"Authorization": f"Bearer {auth._get_authentication_token()}"},
)

## Defining Queries

There are example queries in the `queries` folder that you can use as a starting point. For example, `ClubSessionsFilteredByTimeRange` retrieves all sessions for a club in a specified time period.

## Running Example Queries

For now, let's use the example queries, run them, and print the results.

In [ ]:
from datetime import datetime, timedelta

with open("queries/club_sessions_filtered_by_time_range.graphql", "r") as f:
    club_sessions_filtered_by_time_range = f.read()

# Query for all sessions in the last 30 days
start_time = datetime.now() - timedelta(days=30)
end_time = datetime.now()

last_thirty_days_sessions_response = await client.execute(
    query=club_sessions_filtered_by_time_range,
    variables={"clubId": CLUB_ID, "startTime": start_time, "endTime": end_time},
)

last_thirty_days_sessions_response = client.get_data(last_thirty_days_sessions_response)

print(last_thirty_days_sessions_response)

## Getting Session Details

From the response above, we can see the sessions returned. Now let's get detailed information for a specific session using the session ID from the first session in the response.

In [ ]:
with open("queries/session_details.graphql", "r") as f:
    session_details = f.read()

# Extract the session ID from the first session
first_session_id = last_thirty_days_sessions_response["sessions"][0]["id"]

session_details_response = await client.execute(
    query=session_details, variables={"sessionId": first_session_id}
)

session_details_response = client.get_data(session_details_response)
print(session_details_response)

## Getting Session Metrics

For the same session, you can retrieve configured metrics at different levels:
- Team aggregate level
- Session participation level
- Segment level

In [ ]:
with open("queries/session_metrics.graphql", "r") as f:
    session_metrics = f.read()

# Query for session metrics at all levels
session_metrics_response = await client.execute(
    query=session_metrics, variables={"sessionId": first_session_id}
)

session_metrics_response = client.get_data(session_metrics_response)
print(session_metrics_response)

## Advanced Usage: Raw Data

To query for raw data, you can use the `session_participation_urls` query. This returns URLs for the raw data for a given list of session participation IDs.

We'll use the first session participation ID from the `session_details_response`.

In [ ]:
with open("queries/session_participations_urls.graphql", "r") as f:
    session_participations_urls = f.read()

# Extract the first session participation ID
first_session_participation_id = session_details_response["session"][
    "sessionParticipations"
][0]["id"]

# Query for raw data URLs
session_participation_urls_response = await client.execute(
    query=session_participations_urls,
    variables={"ids": [first_session_participation_id]},
)

session_participation_urls_response = client.get_data(
    session_participation_urls_response
)
print(session_participation_urls_response)

## Downloading Raw Data

The response contains a list of URLs for the raw data for the given session participation ID.

You can download the raw JSON data in two ways:
1. Use the `url_to_csv` function from `raw_data_utils.url_to_csv.py` (shown below)
2. Use the `requests` library directly

The `url_to_csv` function downloads the raw JSON data from the URL and saves it to CSV files:
- One file for GPS data
- One file for IMU acceleration data
- One file for IMU orientation data

The session participation ID is used as a prefix in the filename to differentiate between files.

In [ ]:
import sys
from pathlib import Path

# Add project root to Python path to get access to url_to_csv package
# Only required for notebook example
sys.path.append(str(Path().resolve().parent))
from raw_data_utils.url_to_csv import url_to_csv

# Extract the URL from the first session participation
first_session_participation_url = session_participation_urls_response[
    "sessionParticipations"
][0]["datafiles"][0]["url"]

# Download and convert raw data to CSV files
url_to_csv(first_session_participation_url, first_session_participation_id)

This will create a folder in your root directory with the session participation ID and the three CSV files saved inside.

**Note:** This method of retrieving raw data via the API is currently only available for GPS and IMU data. This method will be deprecated in the future, making way for a simpler method of raw data retrieval where raw LPS data will also be available.